TP 3: Análisis Descriptivo y Predicción de 
Desocupació
Fecha de entrega: 15 de noviembre de 2024
Profesor: Walter Sosa Escudero
Asistente: Tomás Pachecn

La Encuesta Permanente de Hogares (EPH) es un programa nacional de producción sistemática y permanente de indicadores sociales que lleva a cabo el Instituto Nacional de 
Estadística y Censos (INDEC), que permite conocer las características sociodemográfica 
y socioeconómicas de la población. Uno de los indicadores mas valiosos sobre el merca o
laboral que pueden obtenerse con los datos de esta encuesta es la tasa de desocupación.
1. Utilizando información disponible en la pagina del INDEC, expliquen breeem nte
como se identifica a las personas desocupadas.

Ejercicio 2: Entrar a la página de INDEC y descarga la Base de datos.
a) Cada grupo va a trabajar con un aglomerado distinto (nosotros somos grupo 8 - Tucumán). Quédense
con las observaciones correspondientes a su aglomerado y unan ambos trimestres en una sola base.

In [ ]:
import pandas as pd
import os

# Especifica la ruta a la carpeta donde guardaste los archivos
carpeta = "C:\\Users\\marti\\OneDrive\\Documentos\\GitHub\\Machine-Learning\\TP3"

# Define los nombres de los archivos
archivo_dta = "individual_T104.dta"
archivo_xls = "usu_individual_T124.xlsx"

# Carga el archivo .dta
datos_dta = pd.read_stata(os.path.join(carpeta, archivo_dta))

# Carga el archivo .xls
datos_xls = pd.read_excel(os.path.join(carpeta, archivo_xls))

# Verifica los primeros registros de cada archivo para asegurarte de que se cargaron correctamente
print(datos_dta.head())
print(datos_xls.head())

print(datos_dta.columns)
print(datos_xls.columns)

# Filtrar el DataFrame para quedarse solo con el aglomerado "Gran Tucumán - Tafí Viejo"
datos_tucuman04 = datos_dta[datos_dta['aglomerado'] == "Gran Tucumán - Tafí Viejo"]

datos_tucuman24 = datos_xls[datos_xls["AGLOMERADO"]== 29 ]


print(datos_tucuman04.columns)
print(datos_tucuman24.columns)

datos_tucuman04.columns = datos_tucuman04.columns.str.upper()
datos_tucuman24.columns = datos_tucuman24.columns.str.upper()

# Obtener todas las columnas de ambos DataFrames
columnas_comunes = set(datos_tucuman04.columns).union(set(datos_tucuman24.columns))

# Reindexar ambos DataFrames para tener las mismas columnas (y en el mismo orden)
datos_tucuman04 = datos_tucuman04.reindex(columns=columnas_comunes)
datos_tucuman24 = datos_tucuman24.reindex(columns=columnas_comunes)

# Concatenar los DataFrames apilando uno debajo del otro
datos_panel = pd.concat([datos_tucuman04, datos_tucuman24], axis=0)

#Las volvemos a poner en el orden original para que sea mas sencilla su lectura
orden_columnas = datos_xls.columns
datos_panel = datos_panel[orden_columnas]

b) Si hay observaciones con valores que no tienen sentido, descártenlas (por ejemplo, ingresos y edades negativos). Expliquen las decisiones tomadas.

In [ ]:
# Crear una lista para almacenar los resultados
resultados_malos = []

# Iterar por cada columna y obtener los valores únicos y su frecuencia
for columna in datos_panel.columns:
    conteo = datos_panel[columna].value_counts(dropna=False)  # Incluye NaN si hay
    for valor, frecuencia in conteo.items():
        resultados_malos.append({
            'Variable': columna,
            'Valor': valor,
            'Frecuencia': frecuencia
        })

# Convertir la lista de resultados en un DataFrame
resumen_df = pd.DataFrame(resultados_malos)

# Mostrar el DataFrame con todos los valores y frecuencias
import ace_tools as tools; tools.display_dataframe_to_user(name="Resumen de Valores y Frecuencia", dataframe=resumen_df)

# Definir las columnas de ingreso y la columna de edad
columnas_ingreso = ['PP08D1', 'PP08D4', 'PP08F1', 'PP08F2', 'PP08J1', 'PP08J2', 'PP08J3']

# Convertir columnas de ingreso y edad a numérico, forzando errores a NaN
datos_panel['CH06'] = pd.to_numeric(datos_panel['CH06'], errors='coerce')
for columna in columnas_ingreso:
    datos_panel[columna] = pd.to_numeric(datos_panel[columna], errors='coerce')

# Crear un filtro para asegurar que todos los montos de ingreso y edad sean no negativos o NaN
filtro = (datos_panel['CH06'] >= 0) | (datos_panel['CH06'].isna())
for columna in columnas_ingreso:
    filtro &= (datos_panel[columna] >= 0) | (datos_panel[columna].isna())

# Aplicar el filtro al DataFrame
datos_panel_limpio = datos_panel[filtro]

c) Una vez hecha esa limpieza, realicen un gráfico de barras mostrando la composición por sexo para 2004 y 2024. Comenten los resultados

In [ ]:
import matplotlib.pyplot as plt

# Filtrar los datos solo para los años 2004 y 2024
datos_filtrados = datos_panel_limpio[datos_panel_limpio['ANO4'].isin([2004, 2024])]

# Contar la cantidad de personas por año y sexo
conteo_sexo = datos_filtrados.groupby(['ANO4', 'CH04']).size().unstack()

# Crear el gráfico de barras
conteo_sexo.plot(kind='bar', stacked=True)
plt.title("Composición por Sexo en 2004 y 2024")
plt.xlabel("Año")
plt.ylabel("Cantidad de Personas")
plt.legend(["Masculino", "Femenino"], title="Sexo")
plt.xticks(rotation=0)
plt.show()